<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/ai_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="blue">**Training and Tuning**

https://cloud.google.com/vertex-ai/generative-ai/docs/models/tune-models

**Open source:** Ray, Horovod (training), Axolotl (finetune multiple GPU), Unsloth (finetune single GPU), Dask, Spark


Model Garden Tuning: Explore Models https://cloud.google.com/vertex-ai/generative-ai/docs/model-garden/explore-models


Vertex Training: Pricing for training (https://cloud.google.com/vertex-ai/pricing#custom-trained_models) for example SVT for Gemini (https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini-supervised-tuning#supported_models)


Ray on Vertex (managed https://cloud.google.com/vertex-ai/docs/open-source/ray-on-vertex-ai/overview) or Ray on GKE (with KubeRay https://cloud.google.com/kubernetes-engine/docs/add-on/ray-on-gke/concepts/overview), example see Gemma on Ray (https://developers.googleblog.com/en/get-started-with-gemma-on-ray-on-vertex-ai/), Scale AI on Ray on Vertex AI  (https://medium.com/google-cloud/ray-on-vertex-ai-lets-get-it-started-7a9f8360ea25)


Using 🤗 PEFT & bitsandbytes to finetune a LoRa checkpoint

https://huggingface.co/meta-llama/Llama-3.1-8B

https://huggingface.co/models?other=base_model:quantized:meta-llama/Llama-3.1-8B

https://huggingface.co/unsloth/Meta-Llama-3.1-8B-bnb-4bit

https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing#scrollTo=QmUBVEnvCDJv

Original: https://colab.research.google.com/drive/14xo6sj4dARk8lXZbOifHEn1f_70qNAwy?usp=sharing#scrollTo=4iwHGzKBN6wk

Paper: https://arxiv.org/abs/2106.09685

Video: https://www.youtube.com/watch?v=Us5ZFp16PaU&t=393s

https://colab.sandbox.google.com/drive/1CHtIWodkATMxWSkSdaZj60EkFTXymq26?resourcekey=0-YVhm1YZ9excXMSpaTTtxhA#scrollTo=35615fa2-c923-4bfd-b194-5265bbe81ce1

managed tuning OSS



Model Garden

PEFT: https://cloud.google.com/vertex-ai/generative-ai/docs/model-garden/explore-models

Example: Mistral: https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_mistral_peft_tuning.ipynb

Example: LLama3 https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_llama3_finetuning.ipynb

Managed PEFT for open-weight models: pending

Full finetuning: released

DPO - Direct Preference Optimization: directly optimizes the LLM's policy using a dataset of human preferences. https://arxiv.org/pdf/2305.18290.pdf (pending)

Continuous Pretraining: bring your own custom LoRA (end Q2) and tokenspace
